# Initialization

In [2]:
import pandas as pd
import numpy as np

## Read Data

In [3]:
sparse_train_data = np.loadtxt('trainning_data.txt', delimiter=' ', dtype=str)
sparse_test_data = np.loadtxt('test_data.txt', delimiter=' ', dtype=str)

In [4]:
sparse_test_data

array([['0', '7', 'Quality', '1'],
       ['0', '21', 'Quality', '1'],
       ['0', '141', 'Quality', '1'],
       ...,
       ['14892', '1082', 'Quality', '1'],
       ['14897', '21', 'Quality', '1'],
       ['14897', '103', 'Quality', '1']], dtype='<U7')

# Creating a Decision Tree

## For practice get 4 rows from train data

In [20]:
four_samples = full_train_data.sample(4)
four_samples

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
5774,Quality,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5016,Quality,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1605,Spam,1,0,0,0,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5328,Quality,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Create a decision tree using the bootstrapped dataset, but only use a random subset of variables at each step.

in my case I will only use two features, since the only features I have here are the number of spam words.

I need the original dataset as well here

# We want the number of spam words versus non spam words
# Maybe if there is at least 20% spam words then label it as spam?

# Does 20% of spam correctly represent whether its spam or not?

 we can give the function the data and also the difference.

# Gini impurity

Gini Impurity = $ 1 - (P(yes))^2 - (P(no))^2 $

In [16]:
''' gini impurity '''

def gini():
    return 1 - 0 - 0